<a href="https://colab.research.google.com/github/PrashantiSharma/UserProfiler/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install flask pyngrok praw nltk

In [10]:
import praw
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from flask import Flask, render_template_string
from pyngrok import ngrok, conf
conf.get_default().auth_token = "2xJL1xQDkmqOlhVeGTb1PEhkqV6_86jbHMNspeVvtktYhFPUv"
import threading


sia = SentimentIntensityAnalyzer()

# Your Reddit credentials here
reddit = praw.Reddit(
    client_id='jAjrDWn9twXmy5aQg9GYbA',
    client_secret='	2RQFOOTtorNwOJJQsEeLQ-11lK2ZGQ',
    user_agent='PersonaBuilder/0.1 by u/PracticeOk817'
)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>User Persona - u/{{ username }}</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: #fdfdfd;
            margin: 40px;
        }
        .container {
            max-width: 900px;
            margin: auto;
            padding: 25px;
            border: 1px solid #ccc;
            border-radius: 10px;
            background: white;
        }
        h1 {
            color: #e76f51;
        }
        h2 {
            margin-top: 30px;
            color: #264653;
        }
        .section {
            margin-bottom: 20px;
        }
        .bar {
            background-color: #f1f1f1;
            border-radius: 20px;
            padding: 3px;
            margin: 5px 0;
        }
        .bar-fill {
            background-color: #2a9d8f;
            height: 16px;
            border-radius: 20px;
        }
        .quote {
            font-style: italic;
            background: #f8f8f8;
            padding: 10px;
            border-left: 4px solid #e76f51;
            margin-top: 10px;
        }
    </style>
</head>
<body>
<div class="container">
    <h1>User Persona: u/{{ username }}</h1>

    <div class="section">
        <h2>Traits</h2>
        <p><strong>Archetype:</strong> {{ archetype }}<br>
           <strong>Status:</strong> {{ status }}<br>
           <strong>Location:</strong> {{ location }}</p>
    </div>

    <div class="section">
        <h2>Motivations</h2>
        {% for label, percent in motivations.items() %}
            <p>{{ label }}</p>
            <div class="bar">
                <div class="bar-fill" style="width: {{ percent }}%;"></div>
            </div>
        {% endfor %}
    </div>

    <div class="section">
        <h2>Behavior & Habits</h2>
        <ul>
            {% for item in habits %}
                <li>{{ item }}</li>
            {% endfor %}
        </ul>
    </div>

    <div class="section">
        <h2>Frustrations</h2>
        <ul>
            {% for item in frustrations %}
                <li>{{ item }}</li>
            {% endfor %}
        </ul>
    </div>

    <div class="section">
        <h2>Goals & Needs</h2>
        <ul>
            {% for item in goals %}
                <li>{{ item }}</li>
            {% endfor %}
        </ul>
    </div>

    <div class="section">
        <h2>Personality</h2>
        <p><strong>Introvert–Extrovert:</strong> {{ personality['ie'] }}<br>
           <strong>Sensing–Intuition:</strong> {{ personality['sn'] }}<br>
           <strong>Thinking–Feeling:</strong> {{ personality['tf'] }}<br>
           <strong>Judging–Perceiving:</strong> {{ personality['jp'] }}</p>
    </div>

    <div class="quote">
        “{{ quote }}”
    </div>
</div>
</body>
</html>
"""


In [14]:
app = Flask(__name__)

# Define your routes
@app.route('/')
def home():
    return '<h1>Reddit Persona Generator</h1><form action="/persona" method="post">Reddit URL: <input name="url"><input type="submit" value="Generate"></form>'

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

@app.route('/persona', methods=['POST'])
def persona():
    url = request.form['url']
    username = url.strip('/').split('/')[-1]

    redditor = reddit.redditor(username)

    # Fetch posts/comments
    posts, comments = [], []
    try:
        for post in redditor.submissions.new(limit=20):
            posts.append(post.title + " " + post.selftext)
        for comment in redditor.comments.new(limit=20):
            comments.append(comment.body)
    except:
        return f"<h2>Could not fetch data for u/{username}.</h2>"

    # Analyze
    full_text = " ".join(posts + comments).lower()
    motivations = {
        "Convenience": 80 if "quick" in full_text or "fast" in full_text else 50,
        "Wellness": 70 if "fitness" in full_text or "mental" in full_text else 40,
        "Speed": 90 if "instantly" in full_text or "now" in full_text else 60,
        "Preferences": 70,
        "Comfort": 60,
        "Dietary Needs": 30
    }

    habits = [
        "Frequently posts in subreddits related to personal interests.",
        "Engages in community discussions actively.",
        "Comments reflect either helpfulness or curiosity."
    ]
    frustrations = []
    if "can't" in full_text or "don't" in full_text:
        frustrations.append("Expresses confusion or dissatisfaction in discussions.")
    if "hate" in full_text or "annoyed" in full_text:
        frustrations.append("Mentions frustration or irritation.")

    goals = [
        "To get useful feedback and be heard by the community.",
        "To find answers and contribute knowledge.",
        "To feel part of an engaging online space."
    ]

    sentiment_score = sia.polarity_scores(full_text)['compound']
    personality = {
        "ie": "Extrovert" if sentiment_score > 0 else "Introvert",
        "sn": "Intuitive",
        "tf": "Thinking",
        "jp": "Perceiving"
    }

    quote = "I want to spend less time explaining myself and more time being understood."

    return render_template_string(
        html_template,
        username=username,
        archetype="The Explorer",
        status="Unknown",
        location="Unknown",
        motivations=motivations,
        habits=habits,
        frustrations=frustrations if frustrations else ["No clear frustrations found."],
        goals=goals,
        personality=personality,
        quote=quote
    )


# Start ngrok tunnel
def run_app():
    app.run()

thread = threading.Thread(target=run_app)
thread.start()

# Open ngrok tunnel
public_url = ngrok.connect(5000)
print(f"Your app is live at: {public_url}")


 * Serving Flask app '__main__'
 * Debug mode: off
Your app is live at: NgrokTunnel: "https://fc07ee36382c.ngrok-free.app" -> "http://localhost:5000"


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
